In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from utils.load_data import load_data
from utils.preprocessing import preprocess_data
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torchvision import datasets, transforms

%reload_ext autoreload
%autoreload 2

TRENDS_FOLDER = 'data/google_trends/'
GDP_FOLDER = 'data/gdp/'
DATA_PREFIX = 'trends_data_by_topic_'

EPS = 1e-15
SEED = 42

## Loading and Preprocessing Data

### Loading Google Trends

In [2]:
data = load_data()
X, y, countries, y_mean, y_std = preprocess_data(data=data, epsilon=EPS)
X.shape, y.shape

((550, 102), (550,))

In [3]:
X.head()

,date,Expense_average,Research_and_development_average,Capital_expenditure_average,Business_average,Cost_average,Tax_average,Financial_capital_average,Investment_average,Gross_domestic_product_average,...,Artificial_intelligence_average,International_Financial_Reporting_Standards_average,Employment_average,country_Canada,country_Germany,country_Japan,country_Korea,country_Switzerland,country_United Kingdom,country_United States
616,-1.771126,-1.060998,2.871241,-1.696290,1.582391,-0.732737,-0.290201,-0.604001,-0.078562,0.186376,...,-0.395035,-1.710910,1.689417,2.404079,-0.415203,-0.365755,-0.412193,-0.415203,-0.415203,-0.415203
6646,-1.771126,-0.036149,3.231815,0.589877,1.653900,-0.682008,1.399695,0.082269,1.703260,0.763066,...,-0.453614,-0.598496,1.689417,-0.415203,-0.415203,-0.365755,-0.412193,-0.415203,-0.415203,2.404079
6173,-1.771126,-1.009755,2.751050,-1.696290,1.718909,-1.037115,-0.773029,1.073548,2.594171,2.276877,...,-0.277877,0.745672,2.102636,-0.415203,-0.415203,-0.365755,-0.412193,-0.415203,2.404079,-0.415203
3645,-1.771126,-0.446089,2.210188,-1.696290,0.516251,0.636960,0.313333,-0.604001,0.875986,3.790689,...,-0.336456,-1.710910,1.839678,-0.415203,-0.415203,-0.365755,2.421635,-0.415203,-0.415203,-0.415203
5892,-1.771126,2.782184,-2.417180,-1.696290,1.426370,-1.290762,1.327271,-0.604001,2.530535,-3.273765,...,-0.043561,-1.710910,0.900544,-0.415203,-0.415203,-0.365755,-0.412193,2.404079,-0.415203,-0.415203


In [4]:
countries

616             Canada
6646     United States
6173    United Kingdom
3645             Korea
5892       Switzerland
             ...      
856             Canada
6413    United Kingdom
2588           Germany
6132       Switzerland
6886     United States
Name: country, Length: 550, dtype: object

## Simple Prediction Model

### We start with a simple regression model

In [5]:
percent_train = 0.9
number_train = int(len(X) * percent_train)

In [6]:
x_train = X.values[:number_train, :]
y_train = y.values[:number_train]
x_valid = X.values[number_train:, :]
y_valid = y.values[number_train:]
country_train = countries.values[:number_train]
country_valid = countries.values[number_train:]

# Add bias term
# x_train = np.hstack([x_train, np.ones((len(x_train), 1))])
# x_valid = np.hstack([x_valid, np.ones((len(x_valid), 1))])

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_features, 100),
            nn.ReLU(),
            nn.Linear(100, 20),
            nn.ReLU(),
            nn.Linear(20, 1)
        )

    def forward(self, x):
        return self.linear_relu_stack(x)
    
def train_nn(x_train, y_train, x_valid, y_valid, num_epochs=1000, learning_rate=1e-3, weight_decay=1e-5):
    num_features = x_train.shape[1]
    model = NeuralNetwork(num_features=num_features).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    model.train()
    
    x_train_t = torch.tensor(x_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device).unsqueeze(1)
    x_valid_t = torch.tensor(x_valid, dtype=torch.float32).to(device)
    y_valid_t = torch.tensor(y_valid, dtype=torch.float32).to(device).unsqueeze(1)
    
    for t in tqdm(range(num_epochs)):
        model.train()
        y_pred = model(x_train_t)
        loss = loss_fn(y_pred, y_train_t)
        if t % 100 == 99:
            print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    y_pred = model(x_valid_t)
    loss = loss_fn(y_pred, y_valid_t)
    print(f"Validation loss: {loss.item()}")
    
    return model

In [15]:
model = train_nn(x_train, y_train, x_valid, y_valid, num_epochs=10000, learning_rate=1e-3, weight_decay=1e-5)

C:\Users\sebge\AppData\Local\Temp\ipykernel_16608\3944683471.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_t = torch.tensor(x_train, dtype=torch.float32).to(device)
C:\Users\sebge\AppData\Local\Temp\ipykernel_16608\3944683471.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_valid_t = torch.tensor(x_valid, dtype=torch.float32).to(device)


  0%|          | 0/10000 [00:00<?, ?it/s]

99 0.0028402754105627537
199 0.0020668990910053253
299 0.0019034156575798988
399 0.0018080307636409998
499 0.0017317539313808084
599 0.001667567528784275
699 0.0016168984584510326
799 0.0013991605956107378
899 0.0013495302991941571
999 0.0013142366660758853
1099 0.0012091586831957102
1199 0.0011644113110378385
1299 0.001142060849815607
1399 0.0011072884080931544
1499 0.001084288815036416
1599 0.0010567160788923502
1699 0.00101895397529006
1799 0.0012513631954789162
1899 0.0011489186435937881
1999 0.0009951498359441757
2099 0.0010530879953876138
2199 0.0009848717600107193
2299 0.0007704203017055988
2399 0.0007339545409195125
2499 0.0007006237865425646
2599 0.000705586455296725
2699 6.086097710067406e-05
2799 5.322093784343451e-05
2899 9.133150160778314e-05
2999 4.581147004500963e-05
3099 6.345659494400024e-05
3199 4.378070661914535e-05
3299 4.565543349599466e-05
3399 0.00033787020947784185
3499 5.2219336794223636e-05
3599 0.00010864390060305595
3699 3.2594343792879954e-05
3799 4.8903046

In [16]:
# Get the predictions
x_valid = torch.tensor(x_valid, dtype=torch.float32).to(device)
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
y_pred = model(x_valid).clone().detach().cpu().numpy().squeeze()
y_pred_train = model(x_train).clone().detach().cpu().numpy().squeeze()

C:\Users\sebge\AppData\Local\Temp\ipykernel_16608\470602592.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_valid = torch.tensor(x_valid, dtype=torch.float32).to(device)
C:\Users\sebge\AppData\Local\Temp\ipykernel_16608\470602592.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train = torch.tensor(x_train, dtype=torch.float32).to(device)


In [17]:
# Associate the result by country
y_pred_country = pd.DataFrame({'date': X['date'][number_train:], 'country': country_valid, 'y_pred': y_pred, 'y_true': y_valid})
y_pred_train_country = pd.DataFrame({'date': X['date'][:number_train], 'country': country_train, 'y_pred': y_pred_train, 'y_true': y_train})
y_pred_train_country

,date,country,y_pred,y_true
616,-1.771126,Canada,-1.110750,-1.110615
6646,-1.771126,United States,-0.541199,-0.539413
6173,-1.771126,United Kingdom,-1.229731,-1.228650
3645,-1.771126,Korea,-1.833865,-1.837559
5892,-1.771126,Switzerland,-0.633113,-0.636078
...,...,...,...,...
6389,1.366397,United Kingdom,0.579844,0.582981
832,1.366397,Canada,0.933961,0.938508
3527,1.366397,Japan,-0.149573,-0.146885
3861,1.366397,Korea,0.177943,0.173998


In [18]:
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

In [19]:
# Put together the train and the validation set
predictions = pd.concat([y_pred_train_country, y_pred_country])

# Melting the dataframe for better plotting
predictions_melted = predictions.melt(
    id_vars=["date", "country"], value_vars=["y_pred", "y_true"], 
    var_name="Type", value_name="Value"
)



# Function to plot data for the selected country
def plot_by_country(selected_country):
    filtered_data = predictions_melted[predictions_melted["country"] == selected_country]
    cutoff_date = predictions['date'].quantile(percent_train)
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=filtered_data,
        x="date", y="Value", hue="Type", style="Type", markers=True, dashes=False
    )
    plt.title(f"Prediction vs True Values for {selected_country}")
    plt.xlabel("Date")
    plt.ylabel("Values")
    plt.axvline(x=cutoff_date, color='red', linestyle='--', label=f'Validation Start ({percent_train}%)')
    plt.legend(title="Legend")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create a dropdown widget for selecting the country
countries = predictions["country"].unique()
dropdown = widgets.Dropdown(
    options=countries,
    value=countries[0],
    description='Country:'
)

# Use the interact function to link the dropdown with the plot function
interact(plot_by_country, selected_country=dropdown)

plt.show()

interactive(children=(Dropdown(description='Country:', options=('Canada', 'United States', 'United Kingdom', '…